In [ ]:
# Install java
!apt-get update -qq
!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
!java -version

# Install pyspark
!pip install --ignore-installed -q pyspark==2.4.4

# Install Sparknlp+
!pip install --ignore-installed spark-nlp

openjdk version "1.8.0_282"
OpenJDK Runtime Environment (build 1.8.0_282-8u282-b08-0ubuntu1~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.282-b08, mixed mode)
     |████████████████████████████████| 215.7MB 65kB/s 
     |████████████████████████████████| 204kB 15.1MB/s 
     |████████████████████████████████| 143kB 4.3MB/s 


In [ ]:
import pandas as pd
import numpy as np
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
memory = '10g'
pyspark_submit_args = ' --driver-memory ' + memory + ' pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

spark = sparknlp.start()

In [ ]:
attr_df = pd.read_excel('/content/Updated attr dict.xlsx', sheet_name = 4)
attr_df = attr_df[['Attributes List', 'Alternate attribute names', 'Updated Value Attributes']]
topshop_df = pd.read_excel('/content/Women Dresses Web Scraped Data.xlsx', sheet_name = 1)
sample_desc = pd.read_excel('/content/Women Dresses Web Scraped Data.xlsx', sheet_name = 5)

# String matching

In [ ]:
attributenames = [i for i in attr_df['Attributes List'] if i and type(i) == str]
attributenames = [i for i in attributenames if i not in ['Name', 'Price', 'Product Description', 'Decoration', 'With Belt', 'Weight', \
                                                         'Package', 'Style', 'Mechanism', 'Fit'] ]

name_val_dict = {}
for name in attributenames:
  temp = attr_df[attr_df['Attributes List'] == name]
  vals = [i for i in temp['Updated Value Attributes'] if i][0]
  all_vals = vals.split(', ')
  name_val_dict[name] = list(filter(None, list(set([i.lower() for i in all_vals]))))

temp2 = attr_df[attr_df['Attributes List'] == 'Style']
temp2.reset_index(inplace = True)
style_vals = [] #list of dictionaries

for i in temp2['Updated Value Attributes']:
  substyle = i.split(' - ')[0].lower()
  subvals = i.split(' - ')[1].split(', ')
  temp_dict = {}
  temp_dict[substyle] = list(set(i.lower() for i in subvals))
  style_vals.append(temp_dict)

name_val_dict["Style"] = style_vals

def string_matching_extraction(desc):
  ans_list = []
  temp_style = []
  for key, val in name_val_dict.items():
    if key != 'Style':
      for val2 in val:
        if val2 in desc.lower():
          ans_list.append([key, val2]) 
          break
    else:
      for subdict in val: #style has list of dictionaries
        if list(subdict.keys())[0] in desc.lower():
          temp_style.append(list(subdict.keys())[0])
          for subval in list(subdict.values())[0]:
            if subval in desc.lower():
              temp_style.append(subval)
              break
          if len(temp_style) == 1:
            temp_style.append('No Style subvalue')
            break
      if len(temp_style) == 0:
        temp_style.append('No Style value')
        break
  ans_list.append(temp_style)
  return ans_list

In [ ]:
name_val_dict['Size']

In [ ]:
topshop_sample = topshop_df.sample(30)

In [ ]:
topshop_sample['Attr segmented'] = topshop_sample.apply(lambda r : string_matching_extraction(r['Description']), axis = 1)
pd.set_option('display.max_colwidth', 500)
topshop_sample.head(30)

,Name,Price (£),Description,Colour,Attr segmented
313,Black Ruched Sleeve Floral Print Mini Tea Dress,10.00,"This season tap into multiple trends with our multi printed tie dye bodycon mini slip dress. Perfect for creating a youthful spin to fashion, this ensemble is totally modern this season. 91% Polyester, 9% Elastane. Machine wash.Model's height is 5'9 and she wears a size 8.",BLACK,"[[Size, m], [Colour, tan], [Material, polyester], [Silhouette, slip], [Dresses Length, mini], [Pattern Type, ], [Care Instructions / Wash Type, machine wash], [Body Fit, tall], [No Style value]]"
399,Ecru Floral Poplin Tea Dress,15.00,"Add some trending fashion points to your look, day or evening with our all over leopard print jersey mini. This bodycon dress is designed with square neckline for a modern touch to your wardrobe. 92% Viscose, 8% Elastane. Machine wash.Model's height is 5'9 and she wears a size 8.",ECRU,"[[Size, 2], [Colour, tan], [Occasion, day], [Material, jersey], [Dresses Length, mini], [Collar-line, square neck], [Pattern Type, ], [Care Instructions / Wash Type, machine wash], [No Style value]]"
209,Khaki Velvet Hooded Hoodie Mini Dress,15.00,"Modern enthusiasts, this bold black cut and sew midi dress is designed for you. This belted beauty is sure to elevate any edit and add contemporary charm. Whether worn in the daytime or for special evenings, you will add that feminine appeal. We love this ensembles unique approach to dressing up this season! 54% Viscose, 43% Polyester, 3% Elastane. Machine wash.",KHAKI,"[[Size, m], [Colour, black], [Occasion, day], [Material, polyester], [Pattern Type, ], [Care Instructions / Wash Type, machine wash], [No Style value]]"
109,Premium Leisure Black and White Tie Dye Sweatshirt Dress,29.69,"There's something so timeless about a Topshop LBD. This bold black mini dress is designed with long sleeves and a flattering fashionable square neckline with mesh detailing. This ensemble can be elevated with some chunky leather boots. 95% Polyester, 5% Elastane. Machine wash.Model's height is 5'9 and she wears a size 8.",Monochrome,"[[Size, m], [Colour, black], [Material, leather], [Dresses Length, mini], [Collar-line, square neck], [Sleeves Length, long sleeve], [Pattern Type, ], [Care Instructions / Wash Type, machine wash], [No Style value]]"
152,**MATERNITY Black Oversized Roll Neck Knitted Mini Dress,35.99,"Dress your best with Topshop Boutique with our classic summer apple green midi dress. This piece is perfect for daytime strolls or evenings, make your look stylish day or night. 100% Cotton. Machine wash.**PLEASE NOTE THIS ITEM CAN ONLY BE RETURNED VIA POST. STORES ARE UNABLE TO RETURN THESE ITEMS FOR YOU. ALL UK POSTAL RETURNS ARE FREE. INTERNATIONAL POSTAL CHARGES WILL VARY.",BLACK,"[[Size, 0], [Colour, apple green], [Occasion, day], [Material, 100% cotton], [Pattern Type, ], [Season, summer], [Care Instructions / Wash Type, machine wash], [classic, No Style subvalue]]"
292,Denim Puff Sleeve Mini Shirt Dress,15.00,Add some contemporary charm with our split front shirt dress in black and khaki print. It's all about the must-have modern details to elevate style this season. 100% Polyester. Machine wash.Model's height is 5'9 and she wears a size 8.,MID STONE,"[[Size, 0], [Colour, black], [Material, polyester], [Silhouette, split], [Pattern Type, ], [Care Instructions / Wash Type, machine wash], [No Style value]]"
39,Black Denim Tiered Mini Shirt Dress,27.99,"This pink wrapped style tie dye dress is the ultimate mini for making a style statement. We love the trending design, introducing print to your wardrobe. Perfect for evenings, just add some strappy heels. 100% Polyester. Machine wash.Model's height is 5'9 and she wears a size 8.",BLACK,"[[Size, 0], [Colour, pink], [Occasion, evening], [Material, polyester], [Dresses Length, mini], [Pattern Type, ], [Care Instructions / Wash Type, machine wash], [No Style value]]"
81,Black Ribbed Mini Dress,13.99,"Our black textured midi dress with sleeve det

In [ ]:
topshop_sample2 = topshop_sample[['Description', 'Attr segmented']]
topshop_sample2.to_excel('String matching updated attr dict output.xlsx', index = False)

In [ ]:
topshop_ans2 = topshop_sample[['Description', 'Attr segmented']]
pd.set_option('display.max_colwidth', 500)
topshop_ans2.to_excel('Attr Segmented - String Matching.xlsx', index = False)

# SparkNLP experiment

In [ ]:
MODEL_NAME='dependency_typed_conllu'

documentAssembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")
pos = PerceptronModel.pretrained("pos_anc", 'en').setInputCols("document", "token").setOutputCol("pos")
dep_parser = DependencyParserModel.pretrained('dependency_conllu').setInputCols(["document", "pos", "token"]).setOutputCol("dependency")
typed_dep_parser = TypedDependencyParserModel.pretrained('dependency_typed_conllu').setInputCols(["token", "pos", "dependency"]).setOutputCol("dependency_type")
nlpPipeline = Pipeline(stages = [documentAssembler, tokenizer, pos, dep_parser, typed_dep_parser])

pos_anc download started this may take some time.
Approximate size to download 4.3 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.6 MB
[OK!]
dependency_typed_conllu download started this may take some time.
Approximate size to download 257.4 KB
[OK!]


In [ ]:
from tqdm.notebook import tqdm
list_all_df = []
for text in tqdm(list(sample_desc['Product Description'])):
  empty_df = spark.createDataFrame([['']]).toDF("text")
  pipelineModel = nlpPipeline.fit(empty_df)
  df = spark.createDataFrame(pd.DataFrame({"text":[text]}))
  result = pipelineModel.transform(df)
  result_pd = result.select(F.explode(F.arrays_zip('token.result',
                                     'token.begin',
                                     'token.end', 
                                     'pos.result', 
                                     'dependency.result', 
                                                  'dependency.metadata',
                                                  'dependency_type.result')).alias("cols"))\
                                                  .select(F.expr("cols['0']").alias("chunk"),
                                                          F.expr("cols['1']").alias("begin"),
                                                          F.expr("cols['2']").alias("end"),
                                                          F.expr("cols['3']").alias("pos"),
                                                          F.expr("cols['4']").alias("dependency"),
                                                          F.expr("cols['5']").alias("dependency_start"),
                                                          F.expr("cols['6']").alias("dependency_type")).toPandas()
  result_pd = result_pd[['chunk', 'pos', 'dependency_type']]
  result_pd['Description'] = text
  list_all_df.append(result_pd)

In [ ]:
all_df = pd.concat(list_all_df)
joined = pd.merge(all_df, sample_desc, how = 'left', left_on = 'Description', right_on = 'Product Description').drop('Description', axis = 1)

In [ ]:
joined[joined['chunk'].isin('exaggerated sleeve'.split())]

,chunk,pos,dependency_type,Product Description,Relevant keywords/key phrases,Unnamed: 2
188,sleeve,NN,<no-type>,Do you want to show off a pretty floral print ...,"floral print', 'three-quarter sleeve', 'v-neck...","Floral Print,bright colours, three-quarter sle..."
237,sleeve,NN,flat,Bet on the contrast of colours! A big matching...,"contrast of colours', 'big matching floral man...","Floral mandala, colourful, boho,friezesl, ochr..."
469,exaggerated,JJ,<no-type>,Add some drama to the mix with this exaggerate...,"exaggerated sleeve', 'poplin cotton', 'durable...","drama,exaggerated sleeve,dress,ensemble,poplin..."
470,sleeve,NN,<no-type>,Add some drama to the mix with this exaggerate...,"exaggerated sleeve', 'poplin cotton', 'durable...","drama,exaggerated sleeve,dress,ensemble,poplin..."


In [ ]:
temp_words = [i.replace("'", '') for i in list(set(list(joined['Relevant keywords/key phrases'])))]
temp2 = [j for i in temp_words for j in i.split()]

joined_list = []
for i in temp2:
  temp_df = joined[joined['chunk'].isin('')]

joined2.shape

(214, 6)

In [ ]:
joined2['pos'].value_counts()

NN      95
JJ      46
DT      18
NNS     14
IN      12
PRP$    11
VB       5
CD       5
RB       3
VBG      3
VBN      2
Name: pos, dtype: int64

In [ ]:
Love your look, whatever season with this midi dress. This green ensemble is designed with sleeve detailing for a fashion-forward spin on dressing up. 100% Polyester. Machine wash.Model's height is 5'9 and she wears a size 8.

# Stanza - 10 sample descriptions

In [ ]:
!pip install --quiet stanza
import stanza
from stanza.models.common.doc import Document

stanza.download('en')   # This downloads the English models for the neural pipeline
nlp = stanza.Pipeline(lang='en')

     |████████████████████████████████| 286kB 4.3MB/s 


2021-02-22 05:06:15 INFO: Downloading default packages for language: en (English)...
2021-02-22 05:06:37 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-02-22 05:06:37 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-02-22 05:06:37 INFO: Use device: cpu
2021-02-22 05:06:37 INFO: Loading: tokenize
2021-02-22 05:06:37 INFO: Loading: pos
2021-02-22 05:06:37 INFO: Loading: lemma
2021-02-22 05:06:37 INFO: Loading: depparse
2021-02-22 05:06:38 INFO: Loading: sentiment
2021-02-22 05:06:38 INFO: Loading: ner
2021-02-22 05:06:39 INFO: Done loading processors!


In [ ]:
doc = nlp(list(sample_desc['Product Description']))
#print(*[f'word: {word.text}\t \tdeprel: {word.deprel}' for sent in doc.sentences for word in sent.words], sep='\n')
dep_dict = {}
for sent in doc.sentences:
  for word in sent.words:
    dep_dict[word.text] = word.deprel
dep_dict

In [ ]:
#for text in list(sample_desc['Product Description']):
print(sample_desc['Product Description'][5])
doc = nlp(sample_desc['Product Description'][5])
dep_dict = {}
for sent in doc.sentences:
  for word in sent.words:
    dep_dict[word.text] = word.deprel

In [ ]:
#Script to process csv to get dependency tags
desc_100 = pd.read_excel('/content/100 descriptions.xlsx')
desc_100 = desc_100[['Product Description', 'Keywords/phrases']]

def get_dependency_tags(desc, keywords_string):
  desc = desc.replace(',', ' ').replace('-', ' ').replace('/', ' ')
  doc = nlp(desc.lower())
  dep_dict = {}
  for sent in doc.sentences:
    for word in sent.words:
      dep_dict[word.text] = word.deprel
  #print(dep_dict)
  
  key_list = list(dep_dict.keys())
  val_list = list(dep_dict.values())

  keywords_list = [i.lower().replace(',', ' ').replace('-', ' ').replace('/', ' ') for i in keywords_string.split(', ')]
  dep_tags_list = []
  for keyword in keywords_list:
    if len(keyword.split()) == 1:
      dep_tags_list.append( [keyword , dep_dict[keyword]] )
    elif len(keyword.split()) == 2:
      dep_tags_list.append( [keyword, ' '.join( [dep_dict[keyword.split()[0]], dep_dict[keyword.split()[1]]] )])
    elif len(keyword.split()) == 3:
      dep_tags_list.append( [keyword, ' '.join( [dep_dict[keyword.split()[0]], dep_dict[keyword.split()[1]], dep_dict[keyword.split()[2]]] )])
    elif len(keyword.split()) == 4:
      dep_tags_list.append( [keyword, ' '.join( [dep_dict[keyword.split()[0]], dep_dict[keyword.split()[1]], dep_dict[keyword.split()[2]], dep_dict[keyword.split()[3]]] )])
  return dep_tags_list

desc_100['Dep tags'] = desc_100.apply(lambda row : get_dependency_tags(row['Product Description'], row['Keywords/phrases']), axis = 1)
desc_100.head()

,Product Description,Keywords/phrases,Dep tags
0,This ruched all over floral print mini tea dre...,"floral, print, mini, tea, dress, ditsy design,...","[[floral, amod], [print, compound], [mini, com..."
1,Channel a stylish look that always has stateme...,"animal printed, midi, dress, jersey number, po...","[[animal printed, compound amod], [midi, compo..."
2,"We are a fan of the modern monochrome look, wh...","modern, monochrome, floral, long sleeve, mini,...","[[modern, amod], [monochrome, compound], [flor..."
3,Our green all over animal printed ruched long ...,"green, animal printed, long sleeve, mini, dres...","[[green, nsubj], [animal printed, compound amo..."
4,If you are a fan of the animal printed style t...,"animal printed style, tiered dress, ensemble, ...","[[animal printed style, compound amod obj], [t..."


In [ ]:
desc_100.to_csv('100 sample desc with dependency tags.csv', index = False)

In [ ]:
#Counting patterns
pattern_dict = {}
for i in list(desc_100['Dep tags']):
  for sublist in i:
    if sublist[1] in pattern_dict.keys():
      pattern_dict[sublist[1]] += 1
    else:
      pattern_dict[sublist[1]] = 1
pattern_count_df = pd.DataFrame(zip(pattern_dict.keys(), pattern_dict.values()), columns = ['Pattern', 'Count']).sort_values('Count', ascending=False)

In [ ]:
for pattern in list(pattern_count_df[pattern_count_df['Count'] < 5]['Pattern']):
  print(pattern)
  temp = []
  for i in list(desc_100['Dep tags']):
    for sublist in i:
      if sublist[1] == pattern:
        temp.append(sublist[0])
  print(temp)
  print('---')

In [ ]:
def phrases_extraction(depend_list, patterns_list):
  phrases = []
  word_list = depend_list[:][0]
  pat_list = depend_list[:][1]
  #unigrams
  for pattern in pat_list:
    if val in patterns_list:
      phrases.append(key)
  #bigrams
  #for i in range(len(val_list) - 1):
  #  res = ' '.join([val_list[i], val_list[i+1]])
  #  if res in patterns:
  #    phrases.append(' '.join([key_list[i], key_list[i+1]]))
  return phrases

In [ ]:
#function to get keywords given a description
def get_keywords(description, patterns_to_look_for):
  desc = description.replace(',', ' ').replace('-', ' ').replace('/', ' ')
  doc = nlp(desc.lower())
  dep_dict = {}
  for sent in doc.sentences:
    for word in sent.words:
      dep_dict[word.text] = word.deprel

  phrases = []
  key_list = list(dep_dict.keys()) #words
  val_list = list(dep_dict.values()) #dependency tags aka patterns
  #Till here, we got each word with its dependency tag
  #Now we look for all possible patterns i.e. uni/bi/tri/quadgrams
  #unigrams
  for i in range(len(val_list)):
    if val_list[i] in patterns_to_look_for:
      phrases.append(key_list[i])
  #bigrams
  for i in range(len(val_list) - 1):
    res = ' '.join([val_list[i], val_list[i+1]])
    if res in patterns_to_look_for:
      phrases.append(' '.join([key_list[i], key_list[i+1]]))
  #trigrams
  for i in range(len(val_list) - 2):
    res = ' '.join([val_list[i], val_list[i+1], val_list[i+2]])
    if res in patterns_to_look_for:
      phrases.append(' '.join([key_list[i], key_list[i+1], key_list[i+2]]))
  #quadgrams
  for i in range(len(val_list) - 3):
    res = ' '.join([val_list[i], val_list[i+1], val_list[i+2], val_list[i+3]])
    if res in patterns_to_look_for:
      phrases.append(' '.join([key_list[i], key_list[i+1], key_list[i+2], key_list[i+3]]))
  return phrases

In [ ]:
topshop_df.head()

,Name,Price (£),Description,Colour
0,Green Sleeve Detail Midi Dress,25.00,"Love your look, whatever season with this midi...",GREEN
1,IDOL Ruffle Patchwork Print Midi Dress,25.00,Layer up and dress up at the same time with ou...,MULTI
2,Grey Marl Roll Neck Oversized Knitted Jumper D...,35.99,If you are looking to look both cosy and cute ...,GREY MARL
3,Ivory Crew Neck Knitted Mini Dress,35.99,"Introducing Topshop Idol, our most iconic coll...",IVORY
4,IDOL Spot Ruffle Maxi Dress,25.00,This grey cosy ribbed long sleeve midi dress w...,NAVY BLUE


In [ ]:
topshop_sample = topshop_df.sample(30)
patterns = list(pattern_count_df[pattern_count_df['Count'] >= 5]['Pattern'])
topshop_sample['Keywords'] = topshop_sample.apply(lambda row : get_keywords(row['Description'], patterns), axis = 1)
topshop_sample.head()

In [ ]:
attr_seg = []
for keyword_list in list(topshop_sample['Keywords']):
  temp = []
  for keyword in keyword_list:
    temp.append([kmeans_predict(keyword, kmeans), keyword])
  attr_seg.append(temp)

topshop_sample['Attr Segmented'] = attr_seg
topshop_sample = topshop_sample[['Description', 'Keywords', 'Attr Segmented']]
topshop_sample.head()

,Description,Keywords,Attr Segmented
250,Our black and white all over spot full sleeve ...,"[black, white, all, spot, full, sleeve, plunge...","[[Colour, black], [Colour, white], [Season, al..."
268,If you love your ditsy design to make a statem...,"[you, ditsy, design, statement, then, stylish,...","[[Season, you], [Collar-line, ditsy], [Style, ..."
432,Embrace romantic inspired ensembles with this ...,"[embrace, romantic, inspired, ensembles, strik...","[[Fit, embrace], [Style, romantic], [Style, in..."
352,Our black ponte bandeau dress with organza fri...,"[black, ponte, bandeau, dress, organza, frill,...","[[Colour, black], [Season, ponte], [Sleeve Sty..."
416,Get inspired by the festival world with our fa...,"[inspired, festival, world, fashionable, tpu, ...","[[Style, inspired], [Season, festival], [Seaso..."


In [ ]:
topshop_sample.to_excel('30 samples attr segmented.xlsx', index = False)

# K means for clustering attributevalues

Idea - to cluster attribute values and then assign each keyword extracted to a cluster belonging to an attribute 

In [ ]:
attr_df = pd.read_excel('/content/Women Dresses Web Scraped Data.xlsx', sheet_name = 3)
attr_df = attr_df[['Attributes List', 'Alternate attribute names', 'Possible Attribute Values']]

attributenames = [i for i in attr_df['Attributes List'] if i and type(i) == str]
attributenames = [i for i in attributenames if i not in ['Name', 'Price', 'Product Description', 'Decoration', 'With Belt', 'Weight', 'Package'] ]

name_val_dict = {}
for name in attributenames:
  temp = attr_df[attr_df['Attributes List'] == name]
  vals = [i for i in temp['Possible Attribute Values']][0]
  all_vals = vals.split(', ')
  name_val_dict[name] = list(set([i.lower() for i in all_vals]))
name_val_dict['Size'] = list(set([j for i in name_val_dict['Size'] for j in i.split('/')]))

In [ ]:
temp_col1 = []
temp_col2 = []
for key, val in name_val_dict.items():
  for val2 in val:
    temp_col1.append(key)
    temp_col2.append(val2)
attr_df_proper = pd.DataFrame(zip(temp_col1, temp_col2), columns = ['Attr name', 'Attr value'])
print(attr_df_proper.shape)

In [ ]:
#20 clusters for 20 attributes
import tensorflow_hub as hub
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
USE_embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", trainable=False, name='USE_embedding')

In [ ]:
from sklearn.cluster import KMeans
train_embeddings = USE_embed(list(attr_df_proper['Attr name']))
kmeans = KMeans(n_clusters=20, random_state=0).fit(train_embeddings)

In [ ]:
attr_df_proper.loc[:,'Clusters'] = None
attr_df_proper.loc[:,'Clusters'] = kmeans.labels_
pd.set_option('display.max_rows', 1600)

In [ ]:
cluster_codes = attr_df_proper[['Attr name', 'Clusters']].drop_duplicates()
cluster_dict = dict(zip(cluster_codes['Clusters'], cluster_codes['Attr name']))

In [ ]:
def kmeans_predict(text_input, fitted_kmeans):
  tmp_embd = USE_embed([text_input])
  return cluster_dict[fitted_kmeans.predict(tmp_embd)[0]]

In [ ]:
#call kmeans predict on random 100 keywords and see output
from random import sample 

dep_tags_list = list(set([j[0] for i in list(desc_100['Dep tags']) for j in i]))
print(len(dep_tags_list))
sample_keywords = sample(dep_tags_list, 100)

In [ ]:
predictions = []
for keyword in sample_keywords:
  predictions.append(kmeans_predict(keyword, kmeans))

In [ ]:
sample_preds = pd.DataFrame(zip(sample_keywords, predictions), columns = ['Keyword/key phrase', 'Predicted attr name'])
sample_preds.head(10)

,Keyword/key phrase,Predicted attr name
0,front plunges down,Sleeve Style
1,off duty days,Season
2,ethereal,Collar-line
3,flirty,Style
4,everyday,Occasion
5,lightweight,Material
6,warm weather,Season
7,pretty,Style
8,shirt,Sleeve Style
9,breezy,Style


In [ ]:
sample_preds.to_csv('Sample 100 keywords predicted attribute.csv', index = False)

## Data cleaning


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel('/content/online coupons.xlsx')
df.head()

,couponId,title,description,promoCode,url,startDate,endDate,merchantName,merchantLogo,merchantDomain,merchantGroupId,savings,category,rating,primaryCountry,shipToCountries
0,5c5adf53cc87cb2eeb6f28eb6ce05c8e,Worldwide Free Shipping on any order.,Worldwide Free Shipping on any order.,NaN,https://redirect.viglink.com?key=37b874eecfbb3...,2012-01-30,2025-12-31,Tmart.com,https://images.viglink.com/merchant/logo/offic...,tmart.com,263,NaN,NaN,NaN,NaN,NaN
1,d16eda60d3d6cee1e2321fa46bf702bd,Worldwide Free Shipping on any order.,Worldwide Free Shipping on any order.,NaN,https://redirect.viglink.com?key=37b874eecfbb3...,2012-01-30,2025-12-31,TMART LIMITED,https://images.viglink.com/merchant/logo/offic...,tmart.com,263,NaN,NaN,NaN,NaN,NaN
2,94de416dddf6e1c94243980db008a262,Sugar Free Hydration Gnarly Boost Was: $20.95 ...,Sugar Free Hydration Gnarly Boost Was: $20.95 ...,NaN,https://redirect.viglink.com?key=37b874eecfbb3...,2014-01-30,2049-12-31,Gnarly,https://images.viglink.com/merchant/logo/offic...,gognarly.com,38728,NaN,NaN,NaN,NaN,NaN
3,452907407902c59a22c85d69c39e3fc2,2019 Atroz 2 in stock,2019 Full Suspension Atroz 2 Now In Stock! Plu...,NaN,https://redirect.viglink.com?key=37b874eecfbb3...,2018-01-31,2037-12-31,diamondback,https://images.viglink.com/merchant/logo/offic...,diamondback.com,101230,NaN,NaN,NaN,NaN,NaN
4,387c250935f155e974ff715f2dfdd0ac,Free Shipping On Orders Over $49!,Free Shipping On Orders Over $49! at Gander Ou...,NaN,https://redirect.viglink.com?key=37b874eecfbb3...,2018-01-31,2023-01-04,Gander Mountain,https://images.viglink.com/merchant/logo/offic...,ganderoutdoors.com,37069,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.shape

(92985, 16)

In [ ]:
non_permissible = ['Pork', 'Alcohol', 'Gambling', 'Bars', 'Nightclubs', 'Casinos', 'Waterparks', 'Massages', 'Waxing',
                   'Haircut and beauty saloons','Arms and ammunition','Pornography']
all_words = [j for i in non_permissible for j in i.lower().split() if j not in ['and', 'or', 'and/or']] + \
 ['porn', 'spa', 'spas', 'massage', 'bar', 'nightclub', 'gamble', 'liquor', 'casino', 'waterpark', 'wax', 'gun', 'guns', 'shooting', 'rifle', 'pistol'] + \
 ['wine', 'whiskey', 'beer', 'knife', 'CBD', '*VPN*', 'vape']
all_coupons = []
for word in all_words:
  #print('For the word:',word)
  for desc in [i for i in list(sovrn['description']) if type(i) == str]:
    if word.lower() in desc.lower().split():
      #print(desc)
      all_coupons.append(desc)
  #print('')

In [ ]:
finance_terms = []
with open('Finance terms.txt', 'r') as f:
  lines = f.readlines()
  for line in lines:
    temp = line.split(', ')
    for i in temp:
      finance_terms.append(i.lower())

finance_terms = [i.replace('\n', '') for i in finance_terms]
finance_all = list(set([j for i in finance_terms for j in i.split()]))
print(len(finance_terms))
print(len(finance_all))

finance_dict = {}
for word in finance_all:
  val = []
  #print('For the word ',word)
  for desc in [i for i in list(sovrn['description']) if type(i) == str]:
    if word in desc.split():
      val.append(desc)
  finance_dict[word] = val

finance_df = pd.DataFrame(zip(finance_dict.keys(), finance_dict.values()), columns = ['Word', 'Descriptions containing word'])
finance_df.head()

finance_df2 = pd.read_excel('/content/finance terms filter.xlsx')
print(finance_df2.shape)
filtered = list(finance_df2[finance_df2['Boolcol'] != 1.0]['Word'])
print(len(filtered))
filtered_df = finance_df[~finance_df['Word'].isin(filtered)]

302
322
(323, 3)
243


In [ ]:
for i in list(filtered_df['Descriptions containing word']):
  for desc in i:
    all_coupons.append(desc)

In [ ]:
len(all_coupons)

1158

In [ ]:
df_filtered = df[~df['description'].isin(all_coupons)]
print(df_filtered.shape)
merch_to_rem = ['Axos Bank', 'valiantvapes.com']
df_filtered = df_filtered[~df_filtered['merchantName'].isin(merch_to_rem)]
print(df_filtered.shape)

(91854, 16)
(91822, 16)


In [ ]:
df_filtered.to_excel("Filtered coupons.xlsx")